# RAG 베이스 코드 with 업스테이지 API

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [4]:
from rag_utils import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("test-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
test-RAG


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [7]:
import os
import requests
from langchain_upstage import UpstageDocumentParseLoader
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

In [9]:
# 단계 1: 문서 로드(Load Documents)

# DP
file_path = "./data/고졸 성인학습자_사회_54page.pdf"
loader = UpstageDocumentParseLoader(file_path)

pages = loader.load()  # or loader.lazy_load()
for page in pages:
    print(page)

page_content='<header id='0' style='font-size:14px'>25. 1. 14. 오후 10:24</header><br><header id='1' style='font-size:14px'>고졸 성인학습자_사회</header><p id='2' data-category='list'></p><p id='3' data-category='list'></p><p id='4' data-category='list'></p><footer id='5' style='font-size:14px'>218.38.12.36/ebook/ebook/h_society/</footer><br><footer id='6' style='font-size:14px'>1/1</footer>' metadata={'total_pages': 1, 'coordinates': [[{'x': 0.0355, 'y': 0.0173}, {'x': 0.159, 'y': 0.0173}, {'x': 0.159, 'y': 0.0302}, {'x': 0.0355, 'y': 0.0302}], [{'x': 0.5143, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0309}, {'x': 0.5143, 'y': 0.0309}], [{'x': 0.2125, 'y': 0.4215}, {'x': 0.759, 'y': 0.4215}, {'x': 0.759, 'y': 0.4593}, {'x': 0.2125, 'y': 0.4593}], [{'x': 0.2356, 'y': 0.4645}, {'x': 0.7019, 'y': 0.4645}, {'x': 0.7019, 'y': 0.502}, {'x': 0.2356, 'y': 0.502}], [{'x': 0.2093, 'y': 0.5281}, {'x': 0.8371, 'y': 0.5281}, {'x': 0.8371, 'y': 0.6098}, {'x': 0.2093, 'y': 0.6098}], [{'x': 

In [10]:
len(pages)

1

In [11]:
pages[0].page_content

"<header id='0' style='font-size:14px'>25. 1. 14. 오후 10:24</header><br><header id='1' style='font-size:14px'>고졸 성인학습자_사회</header><p id='2' data-category='list'></p><p id='3' data-category='list'></p><p id='4' data-category='list'></p><footer id='5' style='font-size:14px'>218.38.12.36/ebook/ebook/h_society/</footer><br><footer id='6' style='font-size:14px'>1/1</footer>"

In [12]:
# matadata 확인
pages[0].__dict__

{'id': None,
 'metadata': {'total_pages': 1,
  'coordinates': [[{'x': 0.0355, 'y': 0.0173},
    {'x': 0.159, 'y': 0.0173},
    {'x': 0.159, 'y': 0.0302},
    {'x': 0.0355, 'y': 0.0302}],
   [{'x': 0.5143, 'y': 0.0163},
    {'x': 0.6505, 'y': 0.0163},
    {'x': 0.6505, 'y': 0.0309},
    {'x': 0.5143, 'y': 0.0309}],
   [{'x': 0.2125, 'y': 0.4215},
    {'x': 0.759, 'y': 0.4215},
    {'x': 0.759, 'y': 0.4593},
    {'x': 0.2125, 'y': 0.4593}],
   [{'x': 0.2356, 'y': 0.4645},
    {'x': 0.7019, 'y': 0.4645},
    {'x': 0.7019, 'y': 0.502},
    {'x': 0.2356, 'y': 0.502}],
   [{'x': 0.2093, 'y': 0.5281},
    {'x': 0.8371, 'y': 0.5281},
    {'x': 0.8371, 'y': 0.6098},
    {'x': 0.2093, 'y': 0.6098}],
   [{'x': 0.0364, 'y': 0.9706},
    {'x': 0.2465, 'y': 0.9706},
    {'x': 0.2465, 'y': 0.9834},
    {'x': 0.0364, 'y': 0.9834}],
   [{'x': 0.9407, 'y': 0.9716},
    {'x': 0.9604, 'y': 0.9716},
    {'x': 0.9604, 'y': 0.9822},
    {'x': 0.9407, 'y': 0.9822}]]},
 'page_content': "<header id='0' style='f

In [14]:
# OCR
api_key = os.environ['UPSTAGE_API_KEY']
filename = "./data/고졸 성인학습자_사회_54page.pdf"

url = "https://api.upstage.ai/v1/document-ai/ocr"
headers = {"Authorization": f"Bearer {api_key}"}

files = {"document": open(filename, "rb")}
response = requests.post(url, headers=headers, files=files)

print(response.json())

{'apiVersion': '1.1', 'confidence': 0.9788, 'metadata': {'pages': [{'height': 1754, 'page': 1, 'width': 1240}]}, 'mimeType': 'multipart/form-data', 'modelVersion': 'ocr-2.2.1', 'numBilledPages': 1, 'pages': [{'confidence': 0.9788, 'height': 1754, 'id': 0, 'text': '25. 1. 14. 오후 10:24 고졸 성인학습자_사회 \nCHAPTER \n01 산업화 · 도시화에 따른 변화 \n학습목표 \n· 산업화 · 도시화로 나타나는 생활 공간과 생활 양식의 변화를 알 수 있다. \n· 산업화 · 도시화로 나타나는 문제점을 해결하는 방안을 제시할 수 있다. \n출제 \n산업화 · 도시화의 특징과 열섬 현상이 자주 출제된다. \nPoint \n01 산업화 · 도시화에 따른 변화 \n(1) 산업화와 도시화 \n① 산업화 \n㉠ 의미 : 농업 중심의 사회에서 공업, 서비스업 중심으로 변화하는 현상이다. \n특징 \n· 산업화로 인해 공업과 서비스업의 종사자 비중이 늘어난다. \n· 기계화와 분업으로 제품의 대량 생산과 대량 소비가 가능해졌다. \n② 도시화 \n㉠ 의미 : 한 국가 내에서 도시 거주 인구 비율이 높아지고 도시적 생활 양식과 도시적 경관 \n이 확대되는 현상이다. \n특징 : 산업화가 진행되면서 촌락에서 도시로의 인구 이동이 활발해져 도시화의 가속화 \n에 영향을 준다. \n③ 도시화 과정 \n· 도시화율이 낮음 \n초기 단계 \n● 대부분의 인구가 촌락에 분포하고, 1차 산업에 종사함 \n● 산업화를 계기로 이촌 향도 현상이 발생하면서 급격한 도시화가 이루어짐 \n가속화 단계 → 2 · 3차 산업이 발달함 \n● 대부분 개발 도상국은 가속화 단계 진행 중 \n● 높은 도시화율을 보이며, 대도시권이 확대됨 \n· 도시화율의 증가가 점차 느려지거나 정체되어 

In [15]:
response.json()["text"]

'25. 1. 14. 오후 10:24 고졸 성인학습자_사회 \nCHAPTER \n01 산업화 · 도시화에 따른 변화 \n학습목표 \n· 산업화 · 도시화로 나타나는 생활 공간과 생활 양식의 변화를 알 수 있다. \n· 산업화 · 도시화로 나타나는 문제점을 해결하는 방안을 제시할 수 있다. \n출제 \n산업화 · 도시화의 특징과 열섬 현상이 자주 출제된다. \nPoint \n01 산업화 · 도시화에 따른 변화 \n(1) 산업화와 도시화 \n① 산업화 \n㉠ 의미 : 농업 중심의 사회에서 공업, 서비스업 중심으로 변화하는 현상이다. \n특징 \n· 산업화로 인해 공업과 서비스업의 종사자 비중이 늘어난다. \n· 기계화와 분업으로 제품의 대량 생산과 대량 소비가 가능해졌다. \n② 도시화 \n㉠ 의미 : 한 국가 내에서 도시 거주 인구 비율이 높아지고 도시적 생활 양식과 도시적 경관 \n이 확대되는 현상이다. \n특징 : 산업화가 진행되면서 촌락에서 도시로의 인구 이동이 활발해져 도시화의 가속화 \n에 영향을 준다. \n③ 도시화 과정 \n· 도시화율이 낮음 \n초기 단계 \n● 대부분의 인구가 촌락에 분포하고, 1차 산업에 종사함 \n● 산업화를 계기로 이촌 향도 현상이 발생하면서 급격한 도시화가 이루어짐 \n가속화 단계 → 2 · 3차 산업이 발달함 \n● 대부분 개발 도상국은 가속화 단계 진행 중 \n● 높은 도시화율을 보이며, 대도시권이 확대됨 \n· 도시화율의 증가가 점차 느려지거나 정체되어 있음 \n종착 단계 \n● 도시 지역에서 비도시 지역으로 인구가 이동하는 역도시화 현상이 나타나기도 함 \n· 선진국이 해당됨 \n52 PART 03 생활 공간과 사회 \n218.38.12.36/ebook/ebook/h_society/ 1/1'

In [16]:
# OCR을 이용해서 content 수정하기
pages[0].page_content = response.json()["text"]

In [17]:
pages

[Document(metadata={'total_pages': 1, 'coordinates': [[{'x': 0.0355, 'y': 0.0173}, {'x': 0.159, 'y': 0.0173}, {'x': 0.159, 'y': 0.0302}, {'x': 0.0355, 'y': 0.0302}], [{'x': 0.5143, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0309}, {'x': 0.5143, 'y': 0.0309}], [{'x': 0.2125, 'y': 0.4215}, {'x': 0.759, 'y': 0.4215}, {'x': 0.759, 'y': 0.4593}, {'x': 0.2125, 'y': 0.4593}], [{'x': 0.2356, 'y': 0.4645}, {'x': 0.7019, 'y': 0.4645}, {'x': 0.7019, 'y': 0.502}, {'x': 0.2356, 'y': 0.502}], [{'x': 0.2093, 'y': 0.5281}, {'x': 0.8371, 'y': 0.5281}, {'x': 0.8371, 'y': 0.6098}, {'x': 0.2093, 'y': 0.6098}], [{'x': 0.0364, 'y': 0.9706}, {'x': 0.2465, 'y': 0.9706}, {'x': 0.2465, 'y': 0.9834}, {'x': 0.0364, 'y': 0.9834}], [{'x': 0.9407, 'y': 0.9716}, {'x': 0.9604, 'y': 0.9716}, {'x': 0.9604, 'y': 0.9822}, {'x': 0.9407, 'y': 0.9822}]]}, page_content='25. 1. 14. 오후 10:24 고졸 성인학습자_사회 \nCHAPTER \n01 산업화 · 도시화에 따른 변화 \n학습목표 \n· 산업화 · 도시화로 나타나는 생활 공간과 생활 양식의 변화를 알 수 있다. \n· 산업화 · 도시화로 나타나는 문

In [18]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
split_documents = text_splitter.split_documents(pages)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 10


In [19]:
split_documents

[Document(metadata={'total_pages': 1, 'coordinates': [[{'x': 0.0355, 'y': 0.0173}, {'x': 0.159, 'y': 0.0173}, {'x': 0.159, 'y': 0.0302}, {'x': 0.0355, 'y': 0.0302}], [{'x': 0.5143, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0309}, {'x': 0.5143, 'y': 0.0309}], [{'x': 0.2125, 'y': 0.4215}, {'x': 0.759, 'y': 0.4215}, {'x': 0.759, 'y': 0.4593}, {'x': 0.2125, 'y': 0.4593}], [{'x': 0.2356, 'y': 0.4645}, {'x': 0.7019, 'y': 0.4645}, {'x': 0.7019, 'y': 0.502}, {'x': 0.2356, 'y': 0.502}], [{'x': 0.2093, 'y': 0.5281}, {'x': 0.8371, 'y': 0.5281}, {'x': 0.8371, 'y': 0.6098}, {'x': 0.2093, 'y': 0.6098}], [{'x': 0.0364, 'y': 0.9706}, {'x': 0.2465, 'y': 0.9706}, {'x': 0.2465, 'y': 0.9834}, {'x': 0.0364, 'y': 0.9834}], [{'x': 0.9407, 'y': 0.9716}, {'x': 0.9604, 'y': 0.9716}, {'x': 0.9604, 'y': 0.9822}, {'x': 0.9407, 'y': 0.9822}]]}, page_content='25. 1. 14. 오후 10:24 고졸 성인학습자_사회 \nCHAPTER \n01 산업화 · 도시화에 따른 변화 \n학습목표'),
 Document(metadata={'total_pages': 1, 'coordinates': [[{'x': 0.0

In [20]:
# 단계 3: 임베딩(Embedding) 생성
# embeddings = UpstageEmbeddings()

embeddings = UpstageEmbeddings(model="embedding-query")

In [21]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [22]:
vectorstore

In [23]:
for doc in vectorstore.similarity_search("산업화"):
    print(doc.page_content)
    print("+" * 100)

초기 단계 
● 대부분의 인구가 촌락에 분포하고, 1차 산업에 종사함 
● 산업화를 계기로 이촌 향도 현상이 발생하면서 급격한 도시화가 이루어짐
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
25. 1. 14. 오후 10:24 고졸 성인학습자_사회 
CHAPTER 
01 산업화 · 도시화에 따른 변화 
학습목표
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
가속화 단계 → 2 · 3차 산업이 발달함 
● 대부분 개발 도상국은 가속화 단계 진행 중 
● 높은 도시화율을 보이며, 대도시권이 확대됨
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
① 산업화 
㉠ 의미 : 농업 중심의 사회에서 공업, 서비스업 중심으로 변화하는 현상이다. 
특징 
· 산업화로 인해 공업과 서비스업의 종사자 비중이 늘어난다.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [24]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [25]:
retriever

VectorStoreRetriever(tags=['FAISS', 'UpstageEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13694c790>, search_kwargs={})

In [26]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("산업화의 의미는?")

[Document(id='1bc78ce8-e5cc-4466-9531-4efeed5ccd16', metadata={'total_pages': 1, 'coordinates': [[{'x': 0.0355, 'y': 0.0173}, {'x': 0.159, 'y': 0.0173}, {'x': 0.159, 'y': 0.0302}, {'x': 0.0355, 'y': 0.0302}], [{'x': 0.5143, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0163}, {'x': 0.6505, 'y': 0.0309}, {'x': 0.5143, 'y': 0.0309}], [{'x': 0.2125, 'y': 0.4215}, {'x': 0.759, 'y': 0.4215}, {'x': 0.759, 'y': 0.4593}, {'x': 0.2125, 'y': 0.4593}], [{'x': 0.2356, 'y': 0.4645}, {'x': 0.7019, 'y': 0.4645}, {'x': 0.7019, 'y': 0.502}, {'x': 0.2356, 'y': 0.502}], [{'x': 0.2093, 'y': 0.5281}, {'x': 0.8371, 'y': 0.5281}, {'x': 0.8371, 'y': 0.6098}, {'x': 0.2093, 'y': 0.6098}], [{'x': 0.0364, 'y': 0.9706}, {'x': 0.2465, 'y': 0.9706}, {'x': 0.2465, 'y': 0.9834}, {'x': 0.0364, 'y': 0.9834}], [{'x': 0.9407, 'y': 0.9716}, {'x': 0.9604, 'y': 0.9716}, {'x': 0.9604, 'y': 0.9822}, {'x': 0.9407, 'y': 0.9822}]]}, page_content='① 산업화 \n㉠ 의미 : 농업 중심의 사회에서 공업, 서비스업 중심으로 변화하는 현상이다. \n특징 \n· 산업화로 인해 공업과 서비스업의 종사자 비중이 늘어난다.'),

In [27]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [28]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatUpstage(model="solar-pro")

In [29]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
# 생성된 체인에 쿼리(질문)을 입력하고 실행합니다.

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "산업화의 의미는?"
response = chain.invoke(question)
print(response)

농업 중심의 사회에서 공업, 서비스업 중심으로 변화하는 현상이다.


## RAG 사용 안한 경우

In [31]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Answer in Korean.

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatUpstage(model="solar-pro")

# 단계 8: 체인(Chain) 생성
chain = {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "산업화의 의미는?"
response = chain.invoke(question)
print(response)

산업화는 경제 발전의 단계 중 하나로, 농업 중심 사회에서 제조업 중심의 사회로 변화하는 과정을 의미합니다. 이 과정에서 기계화와 대량 생산, 기술 혁신 등이 일어나며, 경제 성장과 함께 사회 구조, 생활 방식, 문화 등 다양한 측면에서 변화가 일어납니다.
